In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import wfdb
import scipy.stats as stats
from scipy.fft import fft, fftfreq

In [2]:
def extract_unique_values_from_folder(folder:str):
    """
    Function to extract just the unique headers of the dat and hea files (removing the file name) 
    """
    unique = set()
    for f in os.listdir(folder):
        unique.add(extract_basename(os.path.join(folder, f)))
        
    return list(unique)

In [3]:
def extract_basename(filename):
    """
    Function to extract just the unique headers of the dat and hea files (removing the file spec) 
    """
    return os.path.splitext(filename)[0]

In [4]:
def extract_info_from_name(filename):
    parsed = {}
    basename = os.path.basename(filename).split(".")[0]
    pattern = r"session(\d+)_participant(\d+)_gesture(\d+)_trial(\d+)"
    match = re.match(pattern, basename)
    parsed['session'] = match.group(1) 
    parsed['participant'] = match.group(2)
    parsed['gesture'] = match.group(3) 
    parsed['trial'] = match.group(4)
    parsed['filename'] = filename
    return parsed

In [5]:
session1 = os.path.abspath("../../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/session1")
session2 = os.path.abspath("../../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/session2")
session3 = os.path.abspath("../../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/session3")
unique_values = extract_unique_values_from_folder(session1)

parsed_data_1 = []
parsed_data_2 = []
parsed_data_3 = []

for participant_dir in os.listdir(session1):
    participant_path = os.path.join(session1, participant_dir)
    if os.path.isdir(participant_path):
        # Loop through files in participant directory
        for file_name in os.listdir(participant_path):
            if file_name.endswith(".hea"):  # Process only .hea files
                file_path = os.path.join(participant_path, file_name)
                if os.path.isfile(file_path):
                    parsed_data_1.append(extract_info_from_name(file_path))

for participant_dir in os.listdir(session2):
    participant_path = os.path.join(session2, participant_dir)
    if os.path.isdir(participant_path):
        # Loop through files in participant directory
        for file_name in os.listdir(participant_path):
            if file_name.endswith(".hea"):  # Process only .hea files
                file_path = os.path.join(participant_path, file_name)
                if os.path.isfile(file_path):
                    parsed_data_2.append(extract_info_from_name(file_path))

for participant_dir in os.listdir(session3):
    participant_path = os.path.join(session3, participant_dir)
    if os.path.isdir(participant_path):
        # Loop through files in participant directory
        for file_name in os.listdir(participant_path):
            if file_name.endswith(".hea"):  # Process only .hea files
                file_path = os.path.join(participant_path, file_name)
                if os.path.isfile(file_path):
                    parsed_data_3.append(extract_info_from_name(file_path))

#parsed_data
df1 = pd.DataFrame(parsed_data_1)
df2 = pd.DataFrame(parsed_data_2)
df3 = pd.DataFrame(parsed_data_3)

df = pd.concat([df1, df2, df3], axis=0)
# df = pd.concat([df1.iloc[:10,:]], axis=0)
df['filename'] = df['filename'].str.replace('.hea', '')
df

,session,participant,gesture,trial,filename
0,1,1,10,1,E:\DS5500-project\data\gesture-recognition-and...
1,1,1,10,2,E:\DS5500-project\data\gesture-recognition-and...
2,1,1,10,3,E:\DS5500-project\data\gesture-recognition-and...
3,1,1,10,4,E:\DS5500-project\data\gesture-recognition-and...
4,1,1,10,5,E:\DS5500-project\data\gesture-recognition-and...
...,...,...,...,...,...
5112,3,9,9,3,E:\DS5500-project\data\gesture-recognition-and...
5113,3,9,9,4,E:\DS5500-project\data\gesture-recognition-and...
5114,3,9,9,5,E:\DS5500-project\data\gesture-recognition-and...
5115,3,9,9,6,E:\DS5500-project\data\gesture-recognition-and...


In [17]:
wave = wfdb.rdrecord(df['filename'].iloc[0])
wave_df = wave.to_dataframe()
wave_df = wave_df.drop(['U1','U2', 'U3','U4'], axis = 1)

In [18]:
wave_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12
0 days 00:00:00,0.018842,0.003025,0.046460,0.104214,0.078568,0.139992,0.093088,0.033105,0.003268,-0.046261,...,0.013156,-0.041120,-0.085215,0.001288,0.099477,0.077323,0.006146,-0.032260,-0.080570,0.010847
0 days 00:00:00.000488281,0.007086,-0.007874,0.004497,0.054350,-0.043084,0.000402,0.053829,0.018269,0.013704,-0.032529,...,-0.021570,-0.043785,-0.120230,-0.029939,0.118181,0.076367,-0.011716,-0.036650,-0.117002,-0.010532
0 days 00:00:00.000976562,-0.018860,-0.037642,-0.040174,-0.003930,-0.175231,-0.169030,-0.003461,-0.003630,-0.009620,-0.050602,...,-0.032417,-0.050653,-0.124954,-0.034229,0.126799,0.077234,-0.018928,-0.040363,-0.123522,-0.021235
0 days 00:00:00.001464843,-0.013056,-0.036159,-0.049285,-0.019746,-0.233636,-0.278486,-0.028711,-0.003322,-0.020864,-0.047258,...,-0.024087,-0.049352,-0.109183,-0.034107,0.109105,0.067106,-0.017590,-0.037744,-0.108100,-0.025145
0 days 00:00:00.001953124,0.025837,0.006250,-0.017654,0.013535,-0.205318,-0.291812,-0.012940,0.019870,0.004719,-0.003508,...,-0.002855,-0.024827,-0.082030,-0.039545,0.057957,0.042862,-0.012414,-0.026509,-0.081828,-0.032091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:00:04.997556035,-0.046034,-0.075884,-0.073877,-0.076665,-0.115671,-0.118026,-0.048888,-0.043507,-0.046553,-0.068948,...,-0.006198,-0.006975,0.028131,0.032132,-0.009935,0.007103,-0.004418,-0.006439,0.023369,0.011748
0 days 00:00:04.998044316,-0.016398,-0.042997,-0.033536,-0.052874,-0.050325,-0.047177,-0.006694,-0.010307,-0.014983,-0.036733,...,-0.006423,-0.009028,0.035122,0.045688,0.011053,0.014663,-0.002744,-0.001974,0.031485,0.029136
0 days 00:00:04.998532597,0.001108,-0.024600,-0.008593,-0.027242,-0.008290,-0.007932,0.023454,0.010110,-0.000144,-0.024079,...,-0.005212,-0.002708,0.042888,0.058652,0.023663,0.012143,-0.002599,0.002870,0.042420,0.039055
0 days 00:00:04.999020878,0.003021,-0.018727,-0.003628,-0.012806,-0.012712,-0.004160,0.032820,0.016069,-0.001028,-0.023260,...,0.001042,0.006685,0.045966,0.057129,0.017105,0.003124,-0.002948,0.004054,0.047149,0.035199


In [19]:
electrodes = list(wave_df.columns)

In [20]:
electrodes

['F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8',
 'W9',
 'W10',
 'W11',
 'W12']

In [21]:
wav_vals = wave_df.values
split_vals = np.array_split(wav_vals, 10)

In [22]:
len(split_vals[0])

1024

In [23]:
new_wav = np.stack(split_vals, axis = 2)

In [24]:
new_wav.shape

(1024, 28, 10)

In [299]:
# df['trial'].unique()

In [300]:
# df['participant'].unique()

In [301]:

# def integrated_EMG(signal):
#     return np.sum(np.abs(signal))

# def mean_absolute_value(signal):
#     return np.mean(np.abs(signal))

# def simple_square_integral(signal):
#     return np.sum(signal**2)

# def root_mean_square(signal):
#     return np.sqrt(np.mean(signal**2))

# def variance(signal):
#     return np.var(signal)

# def myopulse_percentage_rate(signal):
#     return len(np.where(np.diff(np.sign(signal)))[0]) / len(signal)

# def waveform_length(signal):
#     return np.sum(np.abs(np.diff(signal)))

# def difference_variance(signal):
#     return np.var(np.diff(signal))

# def difference_absolute_standard_deviation(signal):
#     return np.std(np.diff(np.abs(signal)))

# def willison_amplitude(signal, threshold=0.1):
#     return np.sum(np.abs(np.diff(signal)) > threshold)

In [6]:
sample = np.random.randn(10,5,3)

In [10]:
th = np.mean(sample, axis = 0, keepdims=True) + 2 * np.std(sample, axis = 0,  keepdims=True)

In [11]:
th.shape

(1, 5, 3)

In [12]:
th

array([[[0.32184779, 2.23455946, 0.86349227],
        [1.47348174, 1.69626434, 2.03424882],
        [2.71472749, 1.54320086, 2.74321919],
        [2.87635381, 1.26924034, 1.14816111],
        [2.04940392, 2.68719615, 2.6525347 ]]])

In [13]:
sample > th

array([[[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False,  True, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, Fals

In [32]:
np.sum(new_wav**2, axis = 0, keepdims=True).shape

(1, 28, 10)

In [15]:
def extract_features_with_chunks(df, num_chunks = 10):
    """
    Extracting the following features:
    'MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
            'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f'
    These will be an array of values for each of the 32 sensors
    """
    #time data
    features = {}
    #time domain features
    time_data = df.values
    split_time_data = np.array_split(time_data, num_chunks)
    time_data = np.stack(split_time_data, axis = 2)
    # print(time_data.shape)
    #Three std seems excessive. This can be changed later
    th = np.mean(time_data, axis = 0, keepdims=True) + 2 * np.std(time_data, axis = 0, keepdims=True)
    features['iemg'] = np.sum(np.abs(time_data), axis=0 , keepdims=True)
    features['mav'] = np.mean(np.abs(time_data), axis=0, keepdims=True)
    features['ssi'] = np.sum(time_data**2, axis=0, keepdims=True)
    # features['myopulse'] = len(np.where(np.diff(np.sign(time_data)))[0]) / len(time_data)
    #ref = https://github.com/tanmaygadgil/DS5500-project/blob/main/code_snippets/python-sebastian/feature_extraction.py
    features['myopulse'] = np.sum(time_data >= th, axis = 0, keepdims=True) / len(time_data)
    features['wflen'] = np.sum(np.abs(np.diff(time_data, axis = 0)), axis=0, keepdims=True)
    features['diffvar'] = np.var(np.diff(time_data, axis = 0), axis = 0, keepdims=True)
    features['dasd'] = np.std(np.diff(np.abs(time_data), axis = 0), axis = 0, keepdims=True)
    #Ref #ref = https://github.com/tanmaygadgil/DS5500-project/blob/main/code_snippets/python-sebastian/feature_extraction.py
    features['willison'] = np.sum(np.abs(np.diff(time_data, axis = 0)) > th, axis = 0, keepdims=True)
    
    features['mean'] = np.mean(time_data, axis = 0, keepdims=True)
    features['min'] = np.min(time_data, axis = 0, keepdims=True)
    features['max'] = np.max(time_data, axis = 0, keepdims=True)
    features['rms'] = np.sqrt(np.mean(time_data**2, axis = 0, keepdims=True))
    features['max'] = np.std(time_data, axis = 0, keepdims=True)
    features['power'] = np.mean(time_data**2, axis = 0, keepdims=True)
    features['peak'] = np.max(np.abs(time_data), axis = 0, keepdims=True)
    features['p2p'] = np.ptp(time_data, axis = 0, keepdims=True)
    features['crest_factor'] = np.max(np.abs(time_data), axis = 0, keepdims=True)/np.sqrt(np.mean(time_data**2, axis = 0))
    features['skew'] = stats.skew(time_data, axis = 0, keepdims=True)
    features['kurtosis'] = stats.kurtosis(time_data, axis = 0, keepdims=True)
    features['form_factor'] =np.sqrt(np.mean(time_data**2, axis = 0, keepdims=True)) / np.mean(time_data, axis = 0)
    features['pulse_indicator'] = np.max(np.abs(time_data), axis = 0, keepdims=True)/np.mean(time_data, axis = 0) 
    
    #Convert to frequency domain
    freq_data = fft(time_data)
    S_f = np.abs(freq_data**2)/len(df)
    features['max_f'] = np.max(S_f, axis = 0, keepdims=True)
    features['sum_f'] = np.sum(S_f, axis = 0, keepdims=True)
    features['mean_f'] = np.mean(S_f, axis = 0, keepdims=True)
    features['var_f'] = np.var(S_f, axis = 0, keepdims=True)
    features['peak_f'] = np.max(np.abs(S_f), axis = 0, keepdims=True)
    features['skew_f'] = stats.skew(S_f, axis = 0, keepdims=True)
    features['kurtosis_f'] = stats.kurtosis(S_f, axis = 0, keepdims=True)
    
    return features
    

In [25]:
features = extract_features_with_chunks(wave_df)

In [26]:
features['ssi'].shape


(1, 28, 10)

In [27]:
list(features.values())

[array([[[ 60.15863149,  47.38807362,  41.87579572,  33.60906348,
           33.93124598,  37.1110746 ,  28.03884034,  30.31749183,
           26.08282912,  26.94361212],
         [ 83.56238109,  62.15798299,  56.74965858,  44.89218591,
           42.77655699,  50.23963366,  36.56883203,  41.03951219,
           34.47406031,  33.79129296],
         [110.04870158,  74.24302141,  67.05763746,  54.55083603,
           47.54854858,  55.92303995,  40.83530015,  46.76925164,
           38.88600455,  36.43757373],
         [105.61885184,  76.39482095,  66.36187144,  51.49756375,
           47.91422782,  47.7389844 ,  38.50021868,  41.77464998,
           37.4519789 ,  36.33830801],
         [ 93.26618462,  79.51945127,  74.9390907 ,  60.91119959,
           56.45397578,  47.91793308,  44.15771968,  51.74541147,
           42.48260105,  45.29500321],
         [ 79.01822617,  74.7192512 ,  69.96592349,  59.43956767,
           54.38664785,  49.7157482 ,  47.18921202,  48.54748592,
           45

In [28]:
f = np.concatenate(list(features.values()), axis = 1, )

In [29]:
for k in features.keys():
    print(f"Key:{k}, shape : {features[k].shape}")

Key:iemg, shape : (1, 28, 10)
Key:mav, shape : (1, 28, 10)
Key:ssi, shape : (1, 28, 10)
Key:myopulse, shape : (1, 28, 10)
Key:wflen, shape : (1, 28, 10)
Key:diffvar, shape : (1, 28, 10)
Key:dasd, shape : (1, 28, 10)
Key:willison, shape : (1, 28, 10)
Key:mean, shape : (1, 28, 10)
Key:min, shape : (1, 28, 10)
Key:max, shape : (1, 28, 10)
Key:rms, shape : (1, 28, 10)
Key:power, shape : (1, 28, 10)
Key:peak, shape : (1, 28, 10)
Key:p2p, shape : (1, 28, 10)
Key:crest_factor, shape : (1, 28, 10)
Key:skew, shape : (1, 28, 10)
Key:kurtosis, shape : (1, 28, 10)
Key:form_factor, shape : (1, 28, 10)
Key:pulse_indicator, shape : (1, 28, 10)
Key:max_f, shape : (1, 28, 10)
Key:sum_f, shape : (1, 28, 10)
Key:mean_f, shape : (1, 28, 10)
Key:var_f, shape : (1, 28, 10)
Key:peak_f, shape : (1, 28, 10)
Key:skew_f, shape : (1, 28, 10)
Key:kurtosis_f, shape : (1, 28, 10)


In [30]:
feature_names = list(features.keys())

In [31]:
feature_names

['iemg',
 'mav',
 'ssi',
 'myopulse',
 'wflen',
 'diffvar',
 'dasd',
 'willison',
 'mean',
 'min',
 'max',
 'rms',
 'power',
 'peak',
 'p2p',
 'crest_factor',
 'skew',
 'kurtosis',
 'form_factor',
 'pulse_indicator',
 'max_f',
 'sum_f',
 'mean_f',
 'var_f',
 'peak_f',
 'skew_f',
 'kurtosis_f']

In [32]:
feature_names = []
for i in features:
    for j in electrodes:
        feature_names.append(f"{i}_{j}")

In [33]:
feature_names

['iemg_F1',
 'iemg_F2',
 'iemg_F3',
 'iemg_F4',
 'iemg_F5',
 'iemg_F6',
 'iemg_F7',
 'iemg_F8',
 'iemg_F9',
 'iemg_F10',
 'iemg_F11',
 'iemg_F12',
 'iemg_F13',
 'iemg_F14',
 'iemg_F15',
 'iemg_F16',
 'iemg_W1',
 'iemg_W2',
 'iemg_W3',
 'iemg_W4',
 'iemg_W5',
 'iemg_W6',
 'iemg_W7',
 'iemg_W8',
 'iemg_W9',
 'iemg_W10',
 'iemg_W11',
 'iemg_W12',
 'mav_F1',
 'mav_F2',
 'mav_F3',
 'mav_F4',
 'mav_F5',
 'mav_F6',
 'mav_F7',
 'mav_F8',
 'mav_F9',
 'mav_F10',
 'mav_F11',
 'mav_F12',
 'mav_F13',
 'mav_F14',
 'mav_F15',
 'mav_F16',
 'mav_W1',
 'mav_W2',
 'mav_W3',
 'mav_W4',
 'mav_W5',
 'mav_W6',
 'mav_W7',
 'mav_W8',
 'mav_W9',
 'mav_W10',
 'mav_W11',
 'mav_W12',
 'ssi_F1',
 'ssi_F2',
 'ssi_F3',
 'ssi_F4',
 'ssi_F5',
 'ssi_F6',
 'ssi_F7',
 'ssi_F8',
 'ssi_F9',
 'ssi_F10',
 'ssi_F11',
 'ssi_F12',
 'ssi_F13',
 'ssi_F14',
 'ssi_F15',
 'ssi_F16',
 'ssi_W1',
 'ssi_W2',
 'ssi_W3',
 'ssi_W4',
 'ssi_W5',
 'ssi_W6',
 'ssi_W7',
 'ssi_W8',
 'ssi_W9',
 'ssi_W10',
 'ssi_W11',
 'ssi_W12',
 'myopulse_F1',
 '

In [35]:
from tqdm import tqdm

In [36]:
gestures = []
participants = []
sessions = []
trials = []
features = []
for i in tqdm(range(len(df))):
    wave = wfdb.rdrecord(df['filename'].iloc[i])
    wave_df = wave.to_dataframe()
    wave_df = wave_df.drop(['U1','U2', 'U3','U4'], axis = 1)
    f = extract_features_with_chunks(wave_df, 10)
    all_features = np.concatenate(list(f.values()), axis = 1)
    
    # if i == 0:
    #     feature_list = all_features
    # else:
    features.append(all_features)

    gestures.append(df['gesture'].iloc[i])
    participants.append(df['participant'].iloc[i])
    sessions.append(df['session'].iloc[i])
    trials.append(df['trial'].iloc[i])

features = np.concatenate(features, axis = 0)

100%|████████████████████████████████████████████████████████████████████████████| 15351/15351 [44:15<00:00,  5.78it/s]


In [37]:
gestures_df = pd.DataFrame({
    "gesture": gestures,
    "participant": participants,
    "session": sessions,
    "trial": trials
})

In [38]:
gestures_df

,gesture,participant,session,trial
0,10,1,1,1
1,10,1,1,2
2,10,1,1,3
3,10,1,1,4
4,10,1,1,5
...,...,...,...,...
15346,9,9,3,3
15347,9,9,3,4
15348,9,9,3,5
15349,9,9,3,6


In [39]:
gestures_df.to_csv("../../data/gestures_list.csv", index = False)

In [41]:
features.shape

(15351, 756, 10)

In [43]:
np.save("../../data/features_10_chunks", features)

In [46]:
with open("../../data/feature_name.txt", "w") as file:
    for l in feature_names:
        file.write(l+"\n")

In [90]:
gestures

['10', '10', '10', '10', '10', '10', '10', '11', '11', '11']

In [91]:
trials

['1', '2', '3', '4', '5', '6', '7', '1', '2', '3']

In [94]:
feature_list.shape

(10, 756, 10)

In [361]:
new_df = pd.DataFrame()
for i in range(len(df)):
    wave = wfdb.rdrecord(df['filename'].iloc[i])
    wave_df = wave.to_dataframe()
    wave_df = wave_df.drop(['U1','U2', 'U3','U4'], axis = 1)
    f = extract_features(wave_df)
    f['gesture'] = df['gesture'].iloc[i]
    f['participant'] = df['participant'].iloc[i]

    e = pd.DataFrame.from_dict(f, orient='index').T
    new_df = new_df.append(e)

new_df

,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,mean,min,...,pulse_indicator,max_f,sum_f,mean_f,var_f,peak_f,skew_f,kurtosis_f,gesture,participant
0,12527.173912,0.043691,1114.276499,4.272852,6654.991878,0.001284,0.032037,6221,"[-3.3865626032371843e-06, -1.8127820411250496e...","[-0.12668372176361933, -0.17513763845865235, -...",...,"[-37407.76019983317, -9928.510290468545, -1250...","[0.0032374708895371988, 0.0024257368417489157,...","[1.1552661529428505, 0.52725591139991, 0.05370...","[0.00011281896024832525, 5.148983509764746e-05...","[4.0759490469740834e-08, 1.009706456877444e-08...","[0.0032374708895371988, 0.0024257368417489157,...","[4.891384248345544, 8.37885999035441, 4.928104...","[40.60522664077152, 129.95181496083688, 39.485...",9,13
0,4995.909239,0.017424,155.117851,5.352344,3172.573858,0.000262,0.013734,66,"[-2.4296390143113294e-05, -2.8655309194855e-05...","[-0.08135848817856524, -0.08836079729179075, -...",...,"[-3348.583377997243, -3083.5750782146765, -517...","[0.0003400682853082485, 8.998830278752999e-05,...","[0.15571759620376374, 0.05185293690611252, 0.0...","[1.5206796504273803e-05, 5.063763369737551e-06...","[5.697483095282278e-10, 5.300152686934246e-11,...","[0.0003400682853082485, 8.998830278752999e-05,...","[3.923348971840542, 3.463509813561349, 2.57487...","[25.723773862588562, 18.487573299564048, 10.25...",1,13
0,2761.574404,0.009632,49.810684,3.706152,1145.086276,0.000043,0.00566,2,"[-1.3949779847863498e-05, -2.4062174569915226e...","[-0.03537256588958768, -0.03968770999791185, -...",...,"[-3634.505130751411, -2337.1056508721085, -392...","[0.00011443323193111958, 0.0001161171355056197...","[0.058170294893887545, 0.023942600676462675, 0...","[5.680692860731206e-06, 2.338144597310808e-06,...","[8.583329426825933e-11, 2.3226795149542194e-11...","[0.00011443323193111958, 0.0001161171355056197...","[3.851142382199922, 8.430328229535146, 6.83667...","[22.740194361564065, 124.17278000709265, 84.37...",17,13
0,15737.169767,0.054887,1582.133054,3.608496,7019.840947,0.001321,0.032525,5706,"[1.1641840060145082e-05, -4.215142767499504e-0...","[-0.37012997533964415, -0.39193950726548765, -...",...,"[31793.081972218017, -92983.68498630782, 20225...","[0.008602481339282963, 0.0017454534757986016, ...","[2.0481391254998913, 0.6776872349640545, 0.051...","[0.00020001358647459875, 6.618039403945844e-05...","[1.2981258482694252e-07, 1.2619632283574979e-0...","[0.008602481339282963, 0.0017454534757986016, ...","[7.161415744031498, 5.1704888382965395, 3.3833...","[101.75185540934247, 45.73876249877211, 17.204...",4,13
0,5779.696596,0.020158,234.385913,2.704883,2196.760459,0.000184,0.012485,363,"[-3.33143730828046e-05, -1.780150324510748e-05...","[-0.14179820646167074, -0.13910536655192493, -...",...,"[-4256.367247530787, -7814.248304572609, -7381...","[0.0011822720162164696, 0.00028295531801698513...","[0.3000423879830985, 0.10784719651565663, 0.00...","[2.9301014451474464e-05, 1.0531952784732094e-0...","[3.1250315537447087e-09, 3.8389478840925754e-1...","[0.0011822720162164696, 0.00028295531801698513...","[6.0074439493810825, 5.048279129084738, 5.7275...","[65.94258739608256, 39.86532288268642, 56.2416...",17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,12265.030151,0.042777,1397.402511,5.405273,8530.122413,0.002867,0.045659,15295,"[1.6498772082619616e-05, 3.2853538128259262e-0...","[-0.17831702695746263, -0.19743424240377513, -...",...,"[10807.896858294565, 60095.275471700355, 16404...","[0.005726731252394037, 0.0032628507498287016, ...","[1.0192742929388048, 0.5427118342247342, 0.053...","[9.953850516980515e-05, 5.2999202561009204e-05...","[5.417596246373904e-08, 1.6793682832055938e-08...","[0.005726731252394037, 0.0032628507498287016, ...","[7.960141908982926, 8.87020938273655, 7.020650...","[112.56002343587748, 134.71770997426515, 80.89...",14,14
0,8162.702779,0.028469,537.170042,5.141504,4462.195065,0.000733,0.022413,3204,"[-1.2198009758019286e-06, 6.58361169842

In [368]:
new_df.reset_index()

,index,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,mean,...,pulse_indicator,max_f,sum_f,mean_f,var_f,peak_f,skew_f,kurtosis_f,gesture,participant
0,0,12527.173912,0.043691,1114.276499,4.272852,6654.991878,0.001284,0.032037,6221,"[-3.3865626032371843e-06, -1.8127820411250496e...",...,"[-37407.76019983317, -9928.510290468545, -1250...","[0.0032374708895371988, 0.0024257368417489157,...","[1.1552661529428505, 0.52725591139991, 0.05370...","[0.00011281896024832525, 5.148983509764746e-05...","[4.0759490469740834e-08, 1.009706456877444e-08...","[0.0032374708895371988, 0.0024257368417489157,...","[4.891384248345544, 8.37885999035441, 4.928104...","[40.60522664077152, 129.95181496083688, 39.485...",9,13
1,0,4995.909239,0.017424,155.117851,5.352344,3172.573858,0.000262,0.013734,66,"[-2.4296390143113294e-05, -2.8655309194855e-05...",...,"[-3348.583377997243, -3083.5750782146765, -517...","[0.0003400682853082485, 8.998830278752999e-05,...","[0.15571759620376374, 0.05185293690611252, 0.0...","[1.5206796504273803e-05, 5.063763369737551e-06...","[5.697483095282278e-10, 5.300152686934246e-11,...","[0.0003400682853082485, 8.998830278752999e-05,...","[3.923348971840542, 3.463509813561349, 2.57487...","[25.723773862588562, 18.487573299564048, 10.25...",1,13
2,0,2761.574404,0.009632,49.810684,3.706152,1145.086276,0.000043,0.00566,2,"[-1.3949779847863498e-05, -2.4062174569915226e...",...,"[-3634.505130751411, -2337.1056508721085, -392...","[0.00011443323193111958, 0.0001161171355056197...","[0.058170294893887545, 0.023942600676462675, 0...","[5.680692860731206e-06, 2.338144597310808e-06,...","[8.583329426825933e-11, 2.3226795149542194e-11...","[0.00011443323193111958, 0.0001161171355056197...","[3.851142382199922, 8.430328229535146, 6.83667...","[22.740194361564065, 124.17278000709265, 84.37...",17,13
3,0,15737.169767,0.054887,1582.133054,3.608496,7019.840947,0.001321,0.032525,5706,"[1.1641840060145082e-05, -4.215142767499504e-0...",...,"[31793.081972218017, -92983.68498630782, 20225...","[0.008602481339282963, 0.0017454534757986016, ...","[2.0481391254998913, 0.6776872349640545, 0.051...","[0.00020001358647459875, 6.618039403945844e-05...","[1.2981258482694252e-07, 1.2619632283574979e-0...","[0.008602481339282963, 0.0017454534757986016, ...","[7.161415744031498, 5.1704888382965395, 3.3833...","[101.75185540934247, 45.73876249877211, 17.204...",4,13
4,0,5779.696596,0.020158,234.385913,2.704883,2196.760459,0.000184,0.012485,363,"[-3.33143730828046e-05, -1.780150324510748e-05...",...,"[-4256.367247530787, -7814.248304572609, -7381...","[0.0011822720162164696, 0.00028295531801698513...","[0.3000423879830985, 0.10784719651565663, 0.00...","[2.9301014451474464e-05, 1.0531952784732094e-0...","[3.1250315537447087e-09, 3.8389478840925754e-1...","[0.0011822720162164696, 0.00028295531801698513...","[6.0074439493810825, 5.048279129084738, 5.7275...","[65.94258739608256, 39.86532288268642, 56.2416...",17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,0,12265.030151,0.042777,1397.402511,5.405273,8530.122413,0.002867,0.045659,15295,"[1.6498772082619616e-05, 3.2853538128259262e-0...",...,"[10807.896858294565, 60095.275471700355, 16404...","[0.005726731252394037, 0.0032628507498287016, ...","[1.0192742929388048, 0.5427118342247342, 0.053...","[9.953850516980515e-05, 5.2999202561009204e-05...","[5.417596246373904e-08, 1.6793682832055938e-08...","[0.005726731252394037, 0.0032628507498287016, ...","[7.960141908982926, 8.87020938273655, 7.020650...","[112.56002343587748, 134.71770997426515, 80.89...",14,14
15347,0,8162.702779,0.028469,537.170042,5.141504,4462.195065,0.000733,0.022413,3204,"[-1.2198009758019286e-06, 6.583611698426713e-0...",...,"[-245081.20113310288, 57295.65733937707, -1134...","[0.003300095685778511, 0.001529226366693726, 4...","[0.6565733455829638, 0.20552095687746663, 0.01...","[6.411849077958631e-05, 2.0070405945065102e-05...","[1.8845802453629284e-08, 2.242111625635349e-09...","[0.00330009

In [370]:
# new_df1 = pd.DataFrame()
# for i in range(10):
#     wave = wfdb.rdrecord(df['filename'].iloc[i])
#     wave_df = wave.to_dataframe()
#     wave_df = wave_df.drop(['U1','U2', 'U3','U4'], axis = 1)
#     f = extract_features(wave_df)
#     f['gesture'] = df['gesture'].iloc[i]
#     f['participant'] = df['participant'].iloc[i]

#     e = pd.DataFrame.from_dict(f, orient='index').T
#     new_df1 = new_df1.append(e, ignore_index = False)

# new_df1.reset_index()

In [369]:
new_df.reset_index().to_csv('/Users/rahul/Downloads/features.csv')

In [371]:
# i = 53
# wave = wfdb.rdrecord(df['filename'].iloc[i])
# wave_df = wave.to_dataframe()
# wave_df = wave_df.drop(['U1','U2', 'U3','U4'], axis = 1)
# f = extract_features(wave_df)
# f['gesture'] = df['gesture'].iloc[i]
# f['participant'] = df['participant'].iloc[i]

# g = pd.DataFrame.from_dict(f, orient='index').T

# # new_df = pd.DataFrame(columns = ['iemg', 'mav', 'ssi', 'myopulse', 'wflen', 'diffvar', 'dasd',
# #        'willison', 'mean', 'min', 'max', 'rms', 'power', 'peak', 'p2p',
# #        'crest_factor', 'skew', 'kurtosis', 'form_factor', 'pulse_indicator',
# #        'max_f', 'sum_f', 'mean_f', 'var_f', 'peak_f', 'skew_f', 'kurtosis_f',
# #        'gesture', 'participant'])
# # new_df.append(e, ignore_index=True)
# # new_df
# e = e.append(g)
# e